In [ ]:
!pip uninstall -y torch torchvision torchaudi
!pip install --quiet transformers accelerate 
!pip install marker-pdf
!pip install torch torchvision torchaudio

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 899.5 kB/s eta 0:00:000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 43.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 24.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━

In [2]:
!pip install pdfplumber
!pip install pypdf
!pip install markdown
!pip install pdfminer.six
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.5 MB/s eta 0:00:00:00:0100:01


In [3]:
import pypdf
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
from pathlib import Path
import markdown
import pdfplumber
import os
import torch 
import torchvision
from datetime import datetime
from IPython.display import FileLink, display
from pdfminer.high_level import extract_text

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Function to check if text contains mathematical symbols
def contains_mathematical_symbols(text):
    """Check if text contains common mathematical symbols"""
    math_symbols = r'[+\-*/=()<>{}\\[\]^√∫∑∏πα-ω]'
    return bool(re.search(math_symbols, text))

def process_pdf_content(pdf_path, keep_pages=True):
    """
    Process PDF content with optional page tracking and markdown formatting
    
    Args:
        pdf_path (str): Path to the PDF file
        keep_pages (bool): Whether to keep page numbers (True) or return formatted markdown (False)
    """
    try:
        # Initialize storage
        text_by_page = {}
        markdown_lines = []
        
        # Extract text with page tracking
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                if not page_text or not page_text.strip():
                    continue
                    
                # Store original text by page
                if keep_pages:
                    text_by_page[page_num] = page_text
                
                # Process for markdown
                lines = page_text.split('\n')
                for line in lines:
                    line = line.strip()
                    if not line:
                        markdown_lines.append('')
                        continue
                        
                    # Format line based on content
                    if line.isupper() and len(line) > 3:
                        markdown_lines.append(f'## {line}')
                    elif contains_mathematical_symbols(line):
                        markdown_lines.append(f'```math\n{line}\n```')
                    else:
                        markdown_lines.append(line)
        
        # Return based on mode
        if keep_pages:
            return text_by_page
        else:
            markdown_text = '\n'.join(markdown_lines)
            
            # Save markdown file to /kaggle/working directory
            filename = os.path.basename(pdf_path)
            base_name = os.path.splitext(filename)[0]
            output_path = f'/kaggle/working/{base_name}.md'
            
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(markdown_text)
                
            print(f"Successfully converted {pdf_path} to markdown")
            print(f"Saved as: {output_path}")
            
            return markdown_text
            
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return {} if keep_pages else ""

In [ ]:
# Equation Identification Function
def identify_equations(text_by_page, model, tokenizer):
    """Use LLM to identify mathematical equations from text"""
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.pad_token_id
    
    equations_by_page = {}
    batch_size = 4
    
    for page_num, text in text_by_page.items():
        page_equations = []
        chunks = [text[i:i+1024] for i in range(0, len(text), 1024)]
        
        for i in range(0, len(chunks), batch_size):
            batch_chunks = chunks[i:i + batch_size]
            batch_prompts = [
                f"""You are a mathematical equation identifier. Identify mathematical equations from the text.
                    Rules:
                    - Return only equations
                    - If no equations found, return 'None'
                    - Return each equation on a new line
                    - Include all equations, even complex ones
                    - Do not include explanations
                    
                    Text: {chunk}
                    
                    Equations:"""
                for chunk in batch_chunks
            ]
            
            inputs = tokenizer(
                batch_prompts,
                padding=True,
                truncation=True,
                max_length=1024,
                return_tensors="pt"
            ).to(model.device)
            
            with torch.cuda.amp.autocast():
                outputs = model.generate(
                    **inputs,
                    max_length=1028,
                    num_return_sequences=1,
                    pad_token_id=tokenizer.pad_token_id,
                    do_sample=False
                )
            
            for output in outputs:
                response = tokenizer.decode(output, skip_special_tokens=True)
                potential_equations = response.split('\n')
                for eq in potential_equations:
                    if eq and eq.strip() != 'None' and contains_mathematical_symbols(eq):
                        page_equations.append(eq.strip())
        
        if page_equations:
            equations_by_page[page_num] = page_equations
    
    return equations_by_page

In [ ]:
#the main function to process the PDF and extract equations
def process_pdf():
    pdf_path = "/kaggle/input/same-shit/next-generation-sample-questions-advanced-algebra-and-functions.pdf"
    
    print("Extracting text with page tracking...")
    text_by_page = process_pdf_content(pdf_path, keep_pages=True)
    
    # Set specific GPU device
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    model_name = "facebook/opt-1.3b"
    
    # Clear GPU memory before loading model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    model_config = {
        "device_map": {"": device},  # Map all model parts to the same device
        "use_cache": True,
        "torch_dtype": torch.float16
    }
    
    try:
        print("Setting up tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            use_fast=True
        )
        tokenizer.pad_token = tokenizer.eos_token
        print("Pad token set to EOS token")
        
        print("Loading model...")
        # Load model directly to specified device
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            **model_config
        ).to(device)
        
        model.config.pad_token_id = tokenizer.pad_token_id
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.backends.cudnn.benchmark = True
        
        print("Model and tokenizer loaded successfully")
        
        print("Identifying equations...")
        equations_by_page = {}
        
        for i, (page_num, text) in enumerate(text_by_page.items(), 1):
            print(f"\rProcessing page {i}/{len(text_by_page)}...", end="")
            page_result = identify_equations({page_num: text}, model, tokenizer)
            if page_result:
                equations_by_page.update(page_result)
        
        print("\nEquation extraction complete!")
        
        # Print only the extracted equations
        total_equations = sum(len(eqs) for eqs in equations_by_page.values())
        print(f"\nFound {total_equations} equations across {len(equations_by_page)} pages\n")
        
        for page_num in sorted(equations_by_page.keys()):
            print(f"\nPage {page_num}:")
            for idx, eq in enumerate(equations_by_page[page_num], 1):
                print(f"{idx}. {eq}")
        
        return equations_by_page
        
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [28]:
# Run the processing
print("Starting PDF processing...")
results = process_pdf()

Starting PDF processing...
Extracting text with page tracking...
Using device: cuda:0
Setting up tokenizer...
Pad token set to EOS token
Loading model...
Model and tokenizer loaded successfully
Identifying equations...
Processing page 1/10...

/tmp/ipykernel_57/1134379063.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing page 2/10...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing page 3/10...

...

Processing page 8/10...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing page 9/10...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processing page 10/10...
Equation extraction complete!

Found 311 equations across 10 pages


Page 1:
1. - Return only equations
2. - If no equations found, return 'None'
3. - Return each equation on a new line
4. - Include all equations, even complex ones
5. - Do not include explanations

Page 2:
1. - Return only equations
2. - If no equations found, return 'None'
3. - Return each equation on a new line
4. - Include all equations, even complex ones
5. - Do not include explanations
6. College Board is a mission-driven not-for-profit organization that connects students to
7. - Return only equations
8. - If no equations found, return 'None'
9. - Return each equation on a new line
10. - Include all equations, even complex ones
11. - Do not include explanations
12. in format and appear discretely (stand alone) across the assessment. The following
13. 01689-087

Page 3:
1. - Return only equations
2. - If no equations found, return 'None'
3. - Return each equation on a new line
4. - Include 